Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 6.75 ms, sys: 563 µs, total: 7.32 ms
Wall time: 10.8 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.1.0.4/7018/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 6 Memory: 7.29 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 248 ms, sys: 34.8 ms, total: 283 ms
Wall time: 2.01 s


[1.8208955640915734,
 1.2774044031921639,
 1.1782996695004608,
 1.9488238827285094,
 1.3117964948675842,
 1.785864645383052,
 1.4168895104615196,
 1.3724268580141823,
 1.3316937314933157,
 1.499129097074494,
 1.4688076536044714,
 1.2652528147972495,
 1.5012648221402722,
 1.0170792205551833,
 1.8731860122624142,
 1.1134749422247796,
 1.0586093208100151,
 1.0780676786154162,
 1.088969420538125,
 1.6198885510146626,
 1.0277514838838329,
 1.6575184216721746,
 1.8485908749238615,
 1.792388837770353,
 1.2612650709298496,
 1.5877843321177934,
 1.3711264074184033,
 1.2422127825240883,
 1.6441872273105806,
 1.7135974156892138,
 1.0166255993056534,
 1.6018167425620198,
 1.9550250548041141,
 1.774127754034728,
 1.0371721115452899,
 1.5195295037453618,
 1.1575350449783939,
 1.4305078881865034,
 1.6188343464147745,
 1.258143868240872,
 1.597328831858845,
 1.8459018026255114,
 1.0852534664549875,
 1.436232260825897,
 1.8269548564981708,
 1.2328397033243128,
 1.1463589642472025,
 1.7994177095023547]

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.8858395637391525,
 1.9425523515001508,
 1.0191632386255687,
 1.0434375669565896,
 1.2839357256598887,
 1.5799858388941026,
 1.709645748836167,
 1.3324893811468106,
 1.1250520706461606,
 1.07983196013894,
 1.4620796493377584,
 1.7474382009136327,
 1.1525236511094137,
 1.4613515090119562,
 1.9356466740883977,
 1.612041445874815,
 1.7412233050673378,
 1.3631851759167644,
 1.1461161054184084,
 1.2184663276036622,
 1.8537283624026748,
 1.1068278113563181,
 1.8860233547691592,
 1.0595353790126394,
 1.1690477756367215,
 1.2701516166770428,
 1.9280712803661835,
 1.0118937747289283,
 1.8815123210015376,
 1.1946173441544288,
 1.3212916158672119,
 1.5043753685432675,
 1.5837336979809593,
 1.9116177209176444,
 1.892641574906739,
 1.3134814029472666,
 1.2962967853714333,
 1.1566627884163092,
 1.7379559363525932,
 1.1485829039792737,
 1.6145663090114144,
 1.4137033756914645,
 1.059241960565101,
 1.8668281456643254,
 1.5961260665028758,
 1.9625208740132456,
 1.25981400229751,
 1.9037972928832512,


3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 229 ms, sys: 31.5 ms, total: 261 ms
Wall time: 2.66 s
